# **DS_PROD-2. Воспроизводимость и контейнеризация приложений**

# 1. Введение

В предыдущем модуле мы рассмотрели базовые принципы работы с моделями на этапе продакшена.

### МЫ НАУЧИЛИСЬ:

* сохранять обученные модели в виде бинарных файлов, а также файлов других форматов, которые поддерживаются языками программирования, отличными от Python;

* локально разворачивать собственный веб-сервис с помощью фреймворка Flask, встраивая в его работу свою модель машинного обучения;

* оптимизировать работу этого веб-сервиса, увеличивая его пропускную способность с помощью таких инструментов, как uWSGI и NGINX.

Однако, как мы уже упомянули в конце прошлого модуля, осталась одна проблема: наш сервис (да и сама модель) работает только на нашем компьютере. Когда мы зальём исходный код на GitHub, а наш коллега Василий склонирует себе репозиторий и попробует запустить сервис на своём «боевом» сервере, нет гарантии, что приложение запустится.

Что, если на сервере Василия нет необходимых зависимостей, таких как Scikit Learn, Flask и так далее? Установить библиотеки несложно, но какие именно их версии нужны Василию? А что, если у него уже стоят некоторые библиотеки, которые вы использовали, но в совершенно другой версии, которая не совместима с вашим приложением?

Кроме того, для обеспечения работы сервиса в связке uWSGI + NGINX Василию придётся проделывать все те манипуляции, которые мы производили в прошлом модуле. Вы уже должны были убедиться, что процесс настройки взаимодействия uWSGI + NGINX + Flask не из лёгких, пусть его и можно обернуть в некоторую инструкцию.

А что, если, ко всему прочему, на сервере Василия стоит другая операционная система, например Windows, которая не поддерживает работу с uWSGI?

Так много вопросов… И у нас есть на них ответы. Проблема, к которой мы подошли, называется **проблемой воспроизводимости**. О ней, как правило, не задумываются новички, однако она часто проявляется на этапе продакшена. Например, мы хотим перенести проект с локальной машины, где вели разработку, на реальный сервер, но из-за разницы в настройках и конфигурациях этот процесс становится болезненным и требует затрат времени, которого всегда остаётся очень мало на этапе продакшена.

В этом модуле мы разберём, как сделать свой код воспроизводимым и как изолировать свой сервис так, чтобы его можно было запустить на любой машине, а самое главное — как сделать это быстро и просто.

## ЦЕЛИ МОДУЛЯ:

* Узнать, что такое воспроизводимость и какими инструментами её можно обеспечить.

* Познакомиться с терминами «виртуализация» и «контейнеризация».

* Научиться создавать виртуальные окружения, изолировать среду разработки, устанавливать и фиксировать зависимости в виртуальных окружениях.

* Познакомиться с инструментом контейнеризации Docker.

* Научиться писать Dockerfile, создавать образы контейнеров, запускать их, а также делиться ими с помощью хостинга docker-образов Docker Hub.

* Создать docker-образ для нашего веб-сервиса и запустить его в контейнере.

С КАКИМ ПРОЕКТОМ БУДЕМ РАБОТАТЬ?

В предыдущем модуле мы написали маленький веб-сервис. В нём функционирует модель машинного обучения, которая выполняет предсказания для данных, поступающих через POST-запросы по эндпоинту '/predict'.

Перед прохождением модуля давайте зафиксируем, как должна выглядеть директория нашего проекта:

```py
├─web
   ├─models
        └─model.pkl
   └─client.py
   └─server.py
```
   
Проект будет располагаться в директории web, в которой находятся файлы:

* `server.py` — содержит интерфейс сервера, реализованный на Flask. В интерфейсе предусмотрено два эндпоинта:
   * `'/'` — корневой, по обращению к которому пользователю возвращается тестовое сообщение;
   * `'/predict'` — предназначенный для обработки POST-запросов. Запросы приходят в виде списка из четырёх чисел в формате `json()`. Результатом выполнения запроса является JSON-словарь с ключом `'prediction'` и значением-предсказанием модели.
   Вы писали функцию для обработки этого запроса в [прошлом модуле](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DST-3.0+28FEB2021/jump_to_id/4be04c897b3c439c932297a7fa603ce1#codeblock).
* `client.py` — скрипт для тестирования POST-запросов на сервер.
* `models` — папка с моделями, в которой находится файл [model.pkl](https://lms-cdn.skillfactory.ru/assets/courseware/v1/edb336d23fdbf18a919f9dbfad55426a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/model.pkl) с моделью.

**Файл server.py**

```py
from flask import Flask, request, jsonify
import pickle
import numpy as np

# загружаем модель из файла
with open('models/model.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)

# создаём приложение
app = Flask(__name__)

@app.route('/')
def index():
    msg = "Тестовое сообщение. Сервер запущен!"
    return msg

@app.route('/predict', methods=['POST'])
def predict():
    #ваш код здесь
    pass

if __name__ == '__main__':
    app.run('localhost', 5000)
```

**Файл client.py**


```py
import requests

if __name__ == '__main__':
    # выполняем POST-запрос на сервер по эндпоинту add с параметром json
    r = requests.post('http://localhost:5000/predict', json=[1, 0, 1, 24])
    # выводим статус запроса
    print('Status code: {}'.format(r.status_code))
    # реализуем обработку результата
    if r.status_code == 200:
        # если запрос выполнен успешно (код обработки=200),
        # выводим результат на экран
        print('Prediction: {}'.format(r.json()['prediction']))
    else:
        # если запрос завершён с кодом, отличным от 200,
        # выводим содержимое ответа
        print(r.text)
```

В этом модуле мы сделаем наш сервис изолированным и поместим его в контейнер.

# 2. Воспроизводимость

Код, разрабатываемый дата-сайентистом, должен быть воспроизводимым в постоянно меняющихся в условиях:

* меняются данные, поступающие на вход;
* трансформируются пайплайны предобработки данных;
* постоянно изменяются гиперпараметры алгоритмов;
* иногда модифицируются или вовсе удаляются алгоритмы популярных библиотек.

В идеале ожидается, что выполнение кода должно приводить к одинаковым результатам в различных условиях. Именно с этой целью в некоторых заданиях вы встречали требование зафиксировать параметр random seed — таким образом, вы уже неоднократно сталкивались с проблемой воспроизводимости на практике.

> **Воспроизводимость результатов** является одним из главных показателей качества модели. Именно поэтому обеспечение воспроизводимости — одна из важнейших проблем в современном ML-инжиниринге.

Мы можем доверять результату, полученному с помощью модели, только когда доподлинно известно, на какой выборке и с каким пайплайном предобработки она была построена. Кроме того, ожидается, что результаты будут консистентными (согласованными с исходными данными), так как необходимо однозначно понимать, какое изменение привело к улучшению или ухудшению результатов.

Наиболее остро вопрос воспроизводимости стоит в случае, когда над одной моделью работает большая DS-команда. Каждый участник команды должен воспроизвести весь пайплайн и получить тот же самый результат. При этом возможные изменения должны доставляться в общий код так же быстро, сохраняя все требования к воспроизводимости.

На схеме приведён классический пайплайн работы над моделью:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/89880afe0d7257758590dcd86a5798bb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DSPROD_md2_2_1.png)

## ИНСТРУМЕНТЫ ОБЕСПЕЧЕНИЯ ВОСПРОИЗВОДИМОСТИ:

* **Версионирование кода.**
    
    Обычно весь пайплайн представлен в виде частей кода, для работы с которым используется знакомая нам распределённая система управления версиями Git в совокупности с хостингом GitHub.

* **Версионирование артефактов.**
    
    В процессе работы над проектом появляются различные артефакты: датасеты, модели, файлы конфигурации и прочее. Для их версионирования обычно используются такие инструменты, как [DVC](https://dvc.org/) и [Sonatype Nexus](https://www.sonatype.com/product-nexus-repository).

* **Виртуализация и контейнеризация.**
    
    Одним из важнейших аспектов воспроизводимости является настройка окружения.

    > **Виртуализация** — это технология изоляции, сохранения состояния и воспроизведения окружения. Благодаря ей вы можете воссоздать на другом компьютере точную копию вашего окружения и тем самым обеспечить воспроизводимость.

    Однако иногда воссоздания окружения недостаточно и необходимо обеспечить не только воспроизводимость версий библиотек, но и воспроизводимость среды выполнения программы — операционной системы, на которой реализован проект. Для этого прибегают к инструментам **контейнеризации**, таким как Docker.

Все перечисленные выше инструменты мы подробно обсудим далее.

* **Управление экспериментами.**
    
    Для каждого запуска обучения необходимо фиксировать настройки гиперпараметров и сохранять их. В этом помогают системы управления экспериментами, например [MLflow](https://mlflow.org/).

Над созданием универсального стандарта воспроизводимости работают многие крупные компании. Это означает, что у каждой компании пока что существуют собственные требования к обеспечению воспроизводимости. Когда вы начнёте работать в DS-команде, ваши коллеги будут требовать от вас их соблюдения. Обычно чем крупнее компания, тем более серьёзные требования к воспроизводимости необходимо соблюдать. Поскольку единого стандарта пока не существует, пути и используемые в разных компаниях инструменты могут кардинально отличаться.

Рекомендуем вам всегда помнить о задаче воспроизводимости результатов модели. Это позволит вам постепенно привить себе тот уровень культуры кода, который будет удовлетворять самые требовательные компании.

# 3. Виртуализация и изолированность. Virtualenv